# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

In [6]:
!pip3 install zeno-client
!pip3 install --upgrade pip
!pip3 install --upgrade bottleneck
!pip3 install langdetect

In [9]:
!/usr/local/bin/python3.11 --version

Python 3.11.4


In [21]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import config

# Initialize a client with our API key.
client = ZenoClient(config.api_key)


# Step 2 - Create a project

In [8]:
project = client.create_project(
    name="Biased Movies Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
        #Uncomment the following line after implementing the function
        ZenoMetric(name="avg length ratio", type="mean", columns=["avg_length_ratio"]),
    ]
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/9a150ec1-ef06-4ed3-80fa-ea3ccb391f88/Biased%20Movies%20Analysis


# Step 3 - Create dataset

In [10]:
import pandas as pd
from langdetect import detect
from collections import Counter
import random

# Sample language detection function
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"

# Create a Pandas DataFrame for the biased dataset of movies
df = pd.DataFrame(
    {
        "id": range(1, 11),  # Using only 10 instances for demonstration
        "text": [
            "I love this movie!",
            "¡Odio esta película!",
            "Ce film est ok.",
            "The acting was great!",
            "The plot was confusing.",
            "I really enjoyed it!",
            "Worst movie ever!",
            "Not bad, but could be better.",
            "A masterpiece!",
            "I didn't like it.",
        ],
        "genre": ["action", "drama", "comedy", "action", "drama", "action", "drama", "comedy", "action", "drama"],
        "year": [random.choice([2020, 2021, 2022]) for _ in range(10)],  # Randomly select year
        "label": ["positive", "negative", "neutral", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Introduce biases in genre
biased_genre = ["action", "action", "drama", "drama", "comedy", "comedy", "comedy", "comedy", "drama", "drama"]
df["biased_genre"] = biased_genre

# Introduce biases in year
biased_year = [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022, 2022]
df["biased_year"] = biased_year

# Add language detection to create a new column
df["detected_language"] = df["text"].apply(detect_language)

# Add additional columns for analysis
df["input_length"] = df["text"].str.len()

#Uncomment the following line
# compare the length of each movie review text to the average length of all texts in the dataset
df["avg_length_ratio"] = df["input_length"] / df["input_length"].mean()



# Step 4 - Upload the Base Dataset


In [11]:

project.upload_dataset(df, id_column="id", data_column="text", label_column="label")



  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


# Step 5 - Upload the AI System Outputs


In [12]:
# For demonstration, let's assume we have system predictions
df_system = pd.DataFrame(
    {
        "output": ["positive", "negative", "negative", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Create an id column to match the base dataset
df_system["id"] = df_system.index + 1

# Measure accuracy for each instance, which is averaged by the ZenoMetric above
df_system["correct"] = (df_system["output"] == df["label"]).astype(int)



# Step 6 - Upload the system outputs


In [13]:
project.upload_system(df_system, name="System A", id_column="id", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


# Slices the analysis

## Genre-based Analysis

- Sentiment distribution across different genres.
- Average review length ratio by genre to see if some genres have longer or shorter reviews on average.

In [15]:
# Genre-based Analysis: Sentiment Distribution and Average Length Ratio by Genre
genre_analysis = df.groupby('genre').agg({
    'label': lambda x: x.value_counts().to_dict(),  # Sentiment distribution
    'avg_length_ratio': 'mean'  # Average length ratio
}).reset_index()

In [16]:
genre_analysis

,genre,label,avg_length_ratio
0,action,{'positive': 4},0.940722
1,comedy,{'neutral': 2},1.134021
2,drama,{'negative': 4},0.992268


**Genre-based Insights**

- Action Genre: Predominantly positive sentiment with an average length ratio close to the overall average, suggesting reviews are of typical length.
- Comedy Genre: Neutral sentiment is prevalent with reviews being slightly longer than average.
- Drama Genre: Skewed towards negative sentiment with reviews having an average length ratio around the overall average.

## Year-based Insights

- How sentiment (positive, negative, neutral) varies by year.
- Changes in average review length ratio over the years.

In [17]:
# Year-based Analysis: Sentiment Distribution and Average Length Ratio by Year
year_analysis = df.groupby('year').agg({
    'label': lambda x: x.value_counts().to_dict(),
    'avg_length_ratio': 'mean'
}).reset_index()

In [19]:
year_analysis

,year,label,avg_length_ratio
0,2020,"{'negative': 2, 'positive': 1}",1.099656
1,2021,"{'positive': 2, 'negative': 1, 'neutral': 1}",1.005155
2,2022,"{'neutral': 1, 'positive': 1, 'negative': 1}",0.893471


**Year-based Insights**

- 2020: A balanced mix of positive and negative sentiments with reviews slightly shorter than average.
- 2021: Leans towards negative sentiments with longer reviews on average, indicating possibly more detailed criticism.
- 2022: Positive and neutral sentiments dominate with shorter reviews, suggesting concise feedback.

## Language-based Analysis
- Sentiment distribution by detected language.
- Average review length ratio by language to understand if reviews in certain languages tend to be longer or shorter.

In [18]:
# Language-based Analysis: Sentiment Distribution and Average Length Ratio by Language
language_analysis = df.groupby('detected_language').agg({
    'label': lambda x: x.value_counts().to_dict(),
    'avg_length_ratio': 'mean'
}).reset_index()

In [20]:
language_analysis

,detected_language,label,avg_length_ratio
0,af,{'negative': 1},0.876289
1,en,"{'positive': 3, 'negative': 2, 'neutral': 1}",1.099656
2,es,{'negative': 1},1.030928
3,fr,{'neutral': 1},0.773196
4,ro,{'positive': 1},0.721649


**Language-based Insights**

- English: A mix of positive, negative, and neutral sentiments with an average length ratio indicating typical review lengths.
- French and Spanish: Limited data points, but French reviews show neutral sentiment with shorter length, and Spanish reviews show negative sentiment with an average length ratio.

From above slicing analysis we can tell more nuanced understanding and decision-making, such as identifying potential biases in review collection or highlighting the need for more balanced data across genres and years.